In [ ]:
!pip install -qU pypdf langchain_community langchain_huggingface langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from sentence_transformers import SentenceTransformer
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
import requests  # For SerpApi requests
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass()


file_path = input("Upload your PDF file path: ")
loader = PyPDFLoader(file_path)
docs = loader.load()
print(f"Loaded {len(docs)} document chunks.")


··········
Upload your PDF file path: /content/hdr_2020_overview_english.pdf
Loaded 36 document chunks.


In [ ]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

texts = [split.page_content for split in splits]
vectors = embedding_model.encode(texts, convert_to_tensor=False)

# Vector Store
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=HuggingFaceEmbeddings()
)
retriever = vectorstore.as_retriever()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def search_web(query):
    params = {
        "q": query,
        "api_key": "ee88a9dae8ac4815cb320fe1712a85a48df5cf2c035ff848b2d5f137fafe87bd",
        "num": 2,
    }
    response = requests.get("https://serpapi.com/search.json", params=params)
    results = response.json()
    web_content = []

    # Parse results to retrieve snippets and URLs
    for result in results.get("organic_results", []):
        snippet = result.get("snippet", "")
        link = result.get("link", "")
        web_content.append(f"{snippet} (Source: {link})")

    return "\n".join(web_content)


In [ ]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following retrieved PDF content and web search results to answer the question. "
    "If you don't know the answer, say that you don't know. "
    "Use concise language."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

llm = ChatGroq(model="llama3-8b-8192")
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [ ]:
from langchain_core.prompts.chat import SystemMessage, HumanMessage

def query_agent(query):
    pdf_results = rag_chain.invoke({"input": query})
    web_results = search_web(query)

    combined_context = f"PDF Context:\n{pdf_results}\n\nWeb Search Context:\n{web_results}"

    messages = [
        SystemMessage(content="You are an assistant for answering questions based on PDF and web data."),
        SystemMessage(content=combined_context),
        HumanMessage(content=query),
    ]

    response = llm.invoke(messages)
    answer_content = response.get("content", "I'm here to help with more questions if needed.")
    return answer_content


In [ ]:
def query_agent(query):
    pdf_results = rag_chain.invoke({"input": query})
    web_results = search_web(query)

    combined_context = f"PDF Context:\n{pdf_results}\n\nWeb Search Context:\n{web_results}"

    messages = [
        SystemMessage(content="You are an assistant for answering questions based on PDF and web data."),
        SystemMessage(content=combined_context),
        HumanMessage(content=query),
    ]

    response = llm.invoke(messages)

    answer_content = response.content if hasattr(response, "content") else "Sorry, I couldn't find an answer."
    return answer_content

while True:
    user_query = input("Enter your question (type 'exit' to stop): ")
    if user_query.lower() == "exit":
        print("Exiting the chat. Goodbye!")
        break

    response = query_agent(user_query)
    print("\nAnswer:", response, "\n")


Enter your question (type 'exit' to stop): hi

Answer: Hi! I'm here to help answer your questions. What would you like to know?

You can ask me anything, and I'll do my best to provide you with accurate information. You can ask me questions about the PDF document or web search context that we have. What's on your mind? 

Enter your question (type 'exit' to stop): what is the document about 

Answer: Based on the provided PDF context, the document appears to be the Human Development Report. The report provides an overview of human development and progress towards achieving the Sustainable Development Goals (SDGs). 

Enter your question (type 'exit' to stop): who are you

Answer: I am an assistant designed to help answer questions based on the content of a provided PDF and web search results. I can provide concise and accurate answers to your questions, but if I'm unsure or don't have enough information, I will say "I don't know." 

